In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_table('/content/amazon_alexa.tsv')
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [ ]:
df.isnull().sum()

rating              0
date                0
variation           0
verified_reviews    0
feedback            0
dtype: int64

In [ ]:
df['variation'].value_counts()

Black  Dot                      516
Charcoal Fabric                 430
Configuration: Fire TV Stick    350
Black  Plus                     270
Black  Show                     265
Black                           261
Black  Spot                     241
White  Dot                      184
Heather Gray Fabric             157
White  Spot                     109
White                            91
Sandstone Fabric                 90
White  Show                      85
White  Plus                      78
Oak Finish                       14
Walnut Finish                     9
Name: variation, dtype: int64

In [ ]:
len(df['variation'].value_counts())

16

In [ ]:
def cleantext(text):
    tokens = word_tokenize(text.lower())
    ftoken = [t for t in tokens if(t.isalpha())]
    stop = stopwords.words("english")
    ctoken = [t for t in ftoken if(t not in stop)]
    lemma = WordNetLemmatizer()
    ltoken = [lemma.lemmatize(t) for t in ctoken]
    return " ".join(ltoken)

In [ ]:
df['verified_reviews']=df['verified_reviews'].apply(cleantext)


In [ ]:
sentlen = []
for sent in df["verified_reviews"]:
  sentlen.append(len(word_tokenize(sent)))

In [ ]:
df["SentLen"] = sentlen
df.head()

,rating,date,variation,verified_reviews,feedback,SentLen
0,5,31-Jul-18,Charcoal Fabric,love echo,1,2
1,5,31-Jul-18,Charcoal Fabric,loved,1,1
2,4,31-Jul-18,Walnut Finish,sometimes playing game answer question correct...,1,17
3,5,31-Jul-18,Charcoal Fabric,lot fun thing yr old learns dinosaur control l...,1,18
4,5,31-Jul-18,Charcoal Fabric,music,1,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x = df['verified_reviews']
y = le.fit_transform(df['variation'])

In [ ]:
np.quantile(sentlen, 0.95)

40.0

In [ ]:
max_len = np.quantile(sentlen, 0.95)
max(df['SentLen'])


245

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.30,random_state=1)


In [ ]:
tok = Tokenizer(char_level=False, split=" ")
tok.fit_on_texts(xtrain)

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

2945

In [ ]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[5, 37, 1741, 180, 1230, 16, 24, 253, 8, 1231, 983],
 [24, 78, 69, 4, 92, 1232],
 [1, 10, 50, 203, 238, 278],
 [],
 [3,
  17,
  42,
  239,
  1742,
  14,
  260,
  21,
  59,
  120,
  1743,
  1744,
  5,
  112,
  69,
  43,
  57,
  6,
  86,
  1],
 [26, 188, 261, 446, 19, 829, 270, 573, 151],
 [133, 87, 40, 69, 204, 24, 53, 240],
 [1, 643, 155, 43, 57, 86, 1, 644, 163, 574, 79],
 [1, 2, 32, 51, 75, 381, 104, 22, 51, 262, 343, 76, 241],
 [90, 4, 724, 10, 41, 1233, 1234],
 [1,
  725,
  143,
  181,
  90,
  90,
  2,
  107,
  69,
  47,
  27,
  164,
  5,
  163,
  344,
  37,
  260,
  984,
  78,
  575,
  138,
  726,
  18,
  124,
  1745,
  1746,
  25,
  152,
  414],
 [182, 18, 53],
 [1],
 [1,
  18,
  98,
  25,
  576,
  577,
  1235,
  62,
  100,
  139,
  645,
  183,
  3,
  415,
  45,
  1236,
  18,
  8,
  985,
  183,
  727,
  481,
  140,
  134,
  73,
  242,
  164,
  44,
  28,
  29,
  243,
  301,
  72],
 [109,
  382,
  271,
  525,
  79,
  11,
  94,
  447,
  20,
  82,
  31,
  38,
  1747,
  27,
  74,
  2

In [ ]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[   0,    0,    0, ...,    8, 1231,  983],
       [   0,    0,    0, ...,    4,   92, 1232],
       [   0,    0,    0, ...,  203,  238,  278],
       ...,
       [   0,    0,    0, ...,  191,    3,   17],
       [   0,    0,    0, ...,    0,    0,  385],
       [   0,    0,    0, ...,    1,    2,   71]], dtype=int32)

In [ ]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [ ]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,60, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=50, epochs=50)

ypred = rnn.predict(seqmattest)

ypred = ypred>0.5

Epoch 1/50
45/45 [==============================] - 3s 23ms/step - loss: -8.8858
Epoch 2/50
45/45 [==============================] - 1s 18ms/step - loss: -57.5990
Epoch 3/50
45/45 [==============================] - 1s 18ms/step - loss: -129.1786
Epoch 4/50
45/45 [==============================] - 1s 18ms/step - loss: -219.1836
Epoch 5/50
45/45 [==============================] - 1s 18ms/step - loss: -331.7848
Epoch 6/50
45/45 [==============================] - 1s 18ms/step - loss: -468.4165
Epoch 7/50
45/45 [==============================] - 1s 18ms/step - loss: -624.4191
Epoch 8/50
45/45 [==============================] - 1s 18ms/step - loss: -792.1309
Epoch 9/50
45/45 [==============================] - 1s 18ms/step - loss: -987.0101
Epoch 10/50
45/45 [==============================] - 1s 18ms/step - loss: -1207.5643
Epoch 11/50
45/45 [==============================] - 1s 18ms/step - loss: -1451.0508
Epoch 12/50
45/45 [==============================] - 1s 18ms/step - loss: -1715.3047
E

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.25      0.04      0.07        75
           1       0.15      0.99      0.26       142
           2       0.00      0.00      0.00        76
           3       0.00      0.00      0.00        97
           4       0.00      0.00      0.00        75
           5       0.00      0.00      0.00       138
           6       0.00      0.00      0.00        98
           7       0.00      0.00      0.00        48
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00        33
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00        32
          12       0.00      0.00      0.00        47
          13       0.00      0.00      0.00        20
          14       0.00      0.00      0.00        27
          15       0.00      0.00      0.00        30

    accuracy                           0.15       945
   macro avg       0.03   